In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn import metrics

In [2]:
df=pd.read_csv("D:/R/Capstone/Confirmed/marketing_campaign_dataset.csv")
df

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,"$16,174.00",6.29,Chicago,Spanish,506,1922,6,Health & Wellness,01-01-2021
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,"$11,566.00",5.61,New York,German,116,7523,7,Fashionistas,02-01-2021
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,"$10,200.00",7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,03-01-2021
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,"$12,724.00",5.55,Miami,Mandarin,217,1820,7,Health & Wellness,04-01-2021
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,"$16,452.00",6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,05-01-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,TechCorp,Display,All Ages,30 days,Google Ads,0.06,"$18,365.00",2.84,Chicago,German,858,5988,1,Tech Enthusiasts,07-12-2021
199996,199997,DataTech Solutions,Email,Men 25-34,15 days,Facebook,0.02,"$8,168.00",4.14,Chicago,Spanish,228,3068,7,Foodies,08-12-2021
199997,199998,DataTech Solutions,Social Media,Men 18-24,45 days,Website,0.05,"$13,397.00",3.25,New York,Mandarin,723,9548,3,Tech Enthusiasts,09-12-2021
199998,199999,Innovate Industries,Influencer,All Ages,30 days,YouTube,0.10,"$18,508.00",3.86,Houston,French,528,2763,1,Foodies,10-12-2021


In [3]:
df["EngagedResponse"] = df['Engagement_Score'].apply(lambda x: 'Yes' if x >=6 else 'no')

In [4]:
df['Conversion']=df['Conversion_Rate'].apply(lambda x:1 if x>=0.07 else 0) 
df['Conversion']

0         0
1         1
2         1
3         1
4         0
         ..
199995    0
199996    0
199997    0
199998    1
199999    0
Name: Conversion, Length: 200000, dtype: int64

In [5]:
df["Engaged"] = df['EngagedResponse'].apply(lambda x: 1 if x=='Yes' else 0)

In [6]:
df_new=df[['Company','Target_Audience','Channel_Used','Customer_Segment','Campaign_Type','Engaged','Conversion']]
df_new.head()

,Company,Target_Audience,Channel_Used,Customer_Segment,Campaign_Type,Engaged,Conversion
0,Innovate Industries,Men 18-24,Google Ads,Health & Wellness,Email,1,0
1,NexGen Systems,Women 35-44,Google Ads,Fashionistas,Email,1,1
2,Alpha Innovations,Men 25-34,YouTube,Outdoor Adventurers,Influencer,0,1
3,DataTech Solutions,All Ages,YouTube,Health & Wellness,Display,1,1
4,NexGen Systems,Men 25-34,YouTube,Health & Wellness,Email,0,0


In [7]:
columns_to_encode = [
    'Company','Target_Audience','Channel_Used','Customer_Segment','Campaign_Type'
]
for col in columns_to_encode:
    df_new[col], _ = pd.factorize(df_new[col])


C:\Users\Tamil Selvan\AppData\Local\Temp\ipykernel_20844\2466592120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col], _ = pd.factorize(df_new[col])
C:\Users\Tamil Selvan\AppData\Local\Temp\ipykernel_20844\2466592120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col], _ = pd.factorize(df_new[col])
C:\Users\Tamil Selvan\AppData\Local\Temp\ipykernel_20844\2466592120.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [8]:
df_new.head()

,Company,Target_Audience,Channel_Used,Customer_Segment,Campaign_Type,Engaged,Conversion
0,0,0,0,0,0,1,0
1,1,1,0,1,0,1,1
2,2,2,1,2,1,0,1
3,3,3,1,0,2,1,1
4,1,2,1,0,0,0,0


In [9]:
X = df_new[['Company','Target_Audience','Channel_Used','Customer_Segment','Campaign_Type']]

In [10]:
y = df_new[['Conversion']]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,train_size=0.7,random_state=42)

In [12]:
X_train.shape

(140000, 5)

In [13]:
X_test.shape

(60000, 5)

In [14]:
y_train.shape

(140000, 1)

In [15]:
y_test.shape

(60000, 1)

In [16]:
model = sm.Logit(y,X)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.671942
         Iterations 4


In [17]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Conversion   No. Observations:               200000
Model:                          Logit   Df Residuals:                   199995
Method:                           MLE   Df Model:                            4
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:               -0.003057
Time:                        15:31:03   Log-Likelihood:            -1.3439e+05
converged:                       True   LL-Null:                   -1.3398e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Company              0.0376      0.003     12.888      0.000       0.032       0.043
Target_Audience      0.0345      0.003     11.792      0.000       0.029       0.040
Channel_Used         0.0352      0.002     14.637      0.000       0.030       0.040
Customer_Segment     0.0420      0.003     14.342      0.000       0.036       0.048
Campaign_Type        0.0411      0.003     14.030      0.000       0.035       0.047
====================================================================================
"""

In [18]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

D:\AN\AnacondaNavigator\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [19]:
y_pred = logreg.predict(X_test)

In [20]:
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.61
Precision: 0.61
F1 Score: 0.76


In [21]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=5  
)

In [22]:
rf_model.fit(X=X_train, y=y_train)

C:\Users\Tamil Selvan\AppData\Local\Temp\ipykernel_20844\3758097093.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X=X_train, y=y_train)


RandomForestClassifier(max_depth=5, n_estimators=200)

In [23]:
rf_model.estimators_

[DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=154382278),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=695214414),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1035224568),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=846757374),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1003058726),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt',
                        random_state=1850506490),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=340711280),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=572702300),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=33549290),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=738527780),
 DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=431015467),
 DecisionTree

In [24]:
rf_model.estimators_[0]

DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=154382278)

In [25]:
rf_model.estimators_[0].predict(X_test)[:10]

D:\AN\AnacondaNavigator\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [26]:
rf_model.estimators_[2].predict(X_test)[:10]

D:\AN\AnacondaNavigator\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [27]:
rf_model.estimators_[3].predict(X_test)[:10]

D:\AN\AnacondaNavigator\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [28]:
rf_model.feature_importances_

array([0.21710329, 0.184994  , 0.20848223, 0.18801165, 0.20140883])

In [29]:
in_sample_preds = rf_model.predict(X_train)
out_sample_preds = rf_model.predict(X_test)

In [30]:
print('In-Sample Accuracy: %0.4f' % accuracy_score(y_train, in_sample_preds))
print('Out-of-Sample Accuracy: %0.4f' % accuracy_score(y_test, out_sample_preds))

In-Sample Accuracy: 0.6074
Out-of-Sample Accuracy: 0.6075


In [31]:
print('In-Sample Precision: %0.4f' % precision_score(y_train, in_sample_preds))
print('Out-of-Sample Precision: %0.4f' % precision_score(y_test, out_sample_preds))

In-Sample Precision: 0.6074
Out-of-Sample Precision: 0.6075


In [32]:
print('In-Sample F1 Score: %0.4f' % f1_score(y_train, in_sample_preds))
print('Out-of-Sample F1 Score: %0.4f' % f1_score(y_test, out_sample_preds))

In-Sample F1 Score: 0.7557
Out-of-Sample F1 Score: 0.7558
